In [12]:
import os
import time
import torch
import easyocr
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
from pathlib import Path
import yaml
from tqdm import tqdm
from typing import Tuple, List

In [6]:
class LicensePlateDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = Path(data_dir)
        self.image_files = []
        self.labels = []
        
        # Get all image files and their labels
        for img_path in self.data_dir.glob('*.*'):
            if img_path.suffix.lower() in ['.png', '.jpg', '.jpeg']:
                self.image_files.append(img_path)
                self.labels.append(img_path.stem)  # Filename is the label
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        # Load image
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert('RGB')
        # Convert to numpy array and normalize
        image = np.array(image) / 255.0
        # Convert to torch tensor and reshape to (C, H, W)
        image = torch.FloatTensor(image).permute(2, 0, 1)
        
        return {
            'image': image,
            'label': self.labels[idx]
        }

def create_dataloaders(base_path, batch_size=32):
    # Create datasets
    train_dataset = LicensePlateDataset(os.path.join(base_path, 'train'))
    val_dataset = LicensePlateDataset(os.path.join(base_path, 'val'))
    test_dataset = LicensePlateDataset(os.path.join(base_path, 'test'))
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    return train_loader, val_loader, test_loader

def fine_tune_easyocr():
    # Initialize EasyOCR reader with pretrained model
    reader = easyocr.Reader(['en'])
    
    # Configuration for fine-tuning
    config = {
        'learning_rate': 1e-4,
        'batch_size': 32,
        'epochs': 50,
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'save_dir': 'fine_tuned_model',
    }
    
    # Create save directory
    os.makedirs(config['save_dir'], exist_ok=True)
    
    # Create dataloaders
    train_loader, val_loader, test_loader = create_dataloaders(
        'dataset',  # Your dataset path
        batch_size=config['batch_size']
    )
    
    # Get the recognition model from EasyOCR
    recognition_model = reader.recognizer.model
    recognition_model.to(config['device'])
    
    # Define optimizer and loss function
    optimizer = optim.Adam(recognition_model.parameters(), lr=config['learning_rate'])
    criterion = nn.CTCLoss(blank=0, reduction='mean')
    
    # Training loop
    best_val_loss = float('inf')
    
    for epoch in range(config['epochs']):
        # Training
        recognition_model.train()
        train_loss = 0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{config["epochs"]}')
        
        for batch in progress_bar:
            images = batch['image'].to(config['device'])
            labels = batch['label']
            
            # Forward pass
            outputs = recognition_model(images)
            
            # Calculate loss
            loss = criterion(outputs, labels)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            progress_bar.set_postfix({'loss': loss.item()})
        
        # Validation
        recognition_model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                images = batch['image'].to(config['device'])
                labels = batch['label']
                outputs = recognition_model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': recognition_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': best_val_loss,
            }, os.path.join(config['save_dir'], 'best_model.pth'))
        
        print(f'Epoch {epoch+1}: Train Loss = {train_loss/len(train_loader):.4f}, '
              f'Val Loss = {val_loss/len(val_loader):.4f}')
def test_pretrained_model(test_dir: str) -> Tuple[float, List[Tuple[str, str, str]]]:
    """
    Test pretrained EasyOCR model on test dataset
    Returns accuracy and list of (filename, true_label, predicted_label)
    """
    print("\nTesting pretrained EasyOCR model...")
    
    # Initialize EasyOCR reader
    reader = easyocr.Reader(['en'])
    
    correct = 0
    total = 0
    results = []
    
    # Get all test images
    test_files = list(Path(test_dir).glob('*.*'))
    test_files = [f for f in test_files if f.suffix.lower() in ['.png', '.jpg', '.jpeg']]
    
    start_time = time.time()
    
    # Process each image
    for img_path in tqdm(test_files, desc="Processing test images"):
        true_label = img_path.stem
        
        # Read image with EasyOCR
        predictions = reader.readtext(str(img_path))
        
        # Get the text with highest confidence if any predictions exist
        if predictions:
            predicted_label = predictions[0][1].upper()  # Convert to uppercase for comparison
        else:
            predicted_label = ""
        
        # Store result
        results.append((img_path.name, true_label, predicted_label))
        
        # Update accuracy
        if predicted_label == true_label:
            correct += 1
        total += 1
    
    accuracy = correct / total if total > 0 else 0
    processing_time = time.time() - start_time
    
    # Print detailed results
    print("\nPretrained Model Results:")
    print(f"Total images processed: {total}")
    print(f"Correct predictions: {correct}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Processing time: {processing_time:.2f} seconds")
    print(f"Average time per image: {processing_time/total:.2f} seconds")
    
    # Print some example predictions
    print("\nSample Predictions (first 10):")
    print("Filename | True Label | Predicted Label")
    print("-" * 50)
    for filename, true, pred in results[:10]:
        print(f"{filename} | {true} | {pred}")
    
    return accuracy, results

def test_fine_tuned_model(model_path, test_loader):
    # Load the fine-tuned model
    reader = easyocr.Reader(['en'])
    recognition_model = reader.recognizer.model
    
    checkpoint = torch.load(model_path)
    recognition_model.load_state_dict(checkpoint['model_state_dict'])
    recognition_model.eval()
    
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in test_loader:
            images = batch['image']
            labels = batch['label']
            outputs = recognition_model(images)
            predictions = outputs.argmax(dim=2)
            
            # Compare predictions with labels
            correct += (predictions == labels).sum().item()
            total += labels.numel()
    
    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.4f}')



In [3]:
# def test_fine_tuned_model(model_path: str, test_dir: str):
#     """
#     Test fine-tuned model and compare with pretrained results
#     """
#     print("\nTesting fine-tuned model...")
    
#     # Load the checkpoint
#     checkpoint = torch.load(model_path)
#     pretrained_accuracy = checkpoint.get('pretrained_accuracy', 0)
    
#     # Initialize reader with fine-tuned model
#     reader = easyocr.Reader(['en'])
#     recognition_model = reader.recognizer.model
#     recognition_model.load_state_dict(checkpoint['model_state_dict'])
#     recognition_model.eval()
    
#     # Test the model
#     accuracy, results = test_pretrained_model(test_dir)
    
#     # Print comparison
#     print("\nModel Comparison:")
#     print(f"Pretrained model accuracy: {pretrained_accuracy:.4f}")
#     print(f"Fine-tuned model accuracy: {accuracy:.4f}")
#     print(f"Improvement: {(accuracy - pretrained_accuracy)*100:.2f}%")
    
#     return accuracy, results

In [14]:
def test_fine_tuned_model(test_dir: str, model_path: str) -> Tuple[float, List[Tuple[str, str, str]]]:
    """
    Test fine-tuned EasyOCR model on test dataset using the same methodology as pretrained testing
    Returns accuracy and list of (filename, true_label, predicted_label)
    """
    print("\nTesting fine-tuned EasyOCR model...")
    
    # Initialize EasyOCR reader with custom model
    reader = easyocr.Reader(
        ['en'],
        model_storage_directory=os.path.dirname(model_path),
        user_network_directory=True,
        recog_network='Transformer'  # Should match your model name
    )
    
    correct = 0
    total = 0
    results = []
    
    # Get all test images
    test_files = list(Path(test_dir).glob('*.*'))
    test_files = [f for f in test_files if f.suffix.lower() in ['.png', '.jpg', '.jpeg']]
    
    start_time = time.time()
    
    # Process each image
    for img_path in tqdm(test_files, desc="Processing test images"):
        true_label = img_path.stem
        
        try:
            # Read image with fine-tuned EasyOCR
            predictions = reader.readtext(str(img_path))
            
            # Get the text with highest confidence if any predictions exist
            if predictions:
                predicted_label = predictions[0][1].upper()  # Convert to uppercase for comparison
                confidence = predictions[0][2]  # Get confidence score
            else:
                predicted_label = ""
                confidence = 0.0
            
            # Store result
            results.append((img_path.name, true_label, predicted_label))
            
            # Update accuracy
            if predicted_label == true_label:
                correct += 1
            total += 1
            
        except Exception as e:
            print(f"Error processing {img_path}: {str(e)}")
            continue
    
    accuracy = correct / total if total > 0 else 0
    processing_time = time.time() - start_time
    
    # Print detailed results
    print("\nFine-tuned Model Results:")
    print(f"Total images processed: {total}")
    print(f"Correct predictions: {correct}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Processing time: {processing_time:.2f} seconds")
    print(f"Average time per image: {processing_time/total:.2f} seconds")
    
    # Print some example predictions
    print("\nSample Predictions (first 10):")
    print("Filename | True Label | Predicted Label")
    print("-" * 50)
    for filename, true, pred in results[:10]:
        print(f"{filename} | {true} | {pred}")
    
    return accuracy, results

In [7]:
if torch.cuda.is_available():
    print(f"CUDA is available. Number of GPUs: {torch.cuda.device_count()}")
else:
    print("CUDA is not available")

CUDA is available. Number of GPUs: 1


In [8]:
_, _, test_loader = create_dataloaders('dataset')

In [15]:
pretrained_accuracy, pretrained_results = test_pretrained_model('dataset/test')
finetuned_accuracy, finetuned_results = test_fine_tuned_model(
    test_dir='dataset/test',
    model_path='Model/english_g2.pth'
)


Testing pretrained EasyOCR model...


Processing test images: 100%|██████████| 73/73 [00:04<00:00, 15.95it/s]


Pretrained Model Results:
Total images processed: 73
Correct predictions: 1
Accuracy: 0.0137
Processing time: 4.58 seconds
Average time per image: 0.06 seconds

Sample Predictions (first 10):
Filename | True Label | Predicted Label
--------------------------------------------------
1033IR.png | 1033IR | N0
2348XR25.jpg | 2348XR25 | D210
34EN9199.png | 34EN9199 | TRI
381ATK83_1.png | 381ATK83_1 | 381 ATK 83
55SG53.jpg | 55SG53 | 55
5658YA22.jpg | 5658YA22 | 6658Y42
5888881.png | 5888881 | 58088881
5B40001.png | 5B40001 | 5B4
75N1960G.png | 75N1960G | F
8427XX29.jpg | 8427XX29 | [427 XX29

Testing fine-tuned EasyOCR model...


TypeError: expected str, bytes or os.PathLike object, not bool

In [19]:
    reader = easyocr.Reader(
        ['en'],
        model_storage_directory=os.path.dirname(r"D:\Project\License_plate\Model"),
        user_network_directory=True,
        recog_network='Transformer'  # Should match your model name
    )

TypeError: expected str, bytes or os.PathLike object, not bool